In [1]:
%pip install -qU modal-client

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

os.environ["MODEL_NAME"] = "ggml-mpt-7b-instruct.bin"

In [3]:
import modal
import os

stub = modal.Stub("gpt4all")


def download_model():
    import gpt4all
    import os
    # you can use any model from https://gpt4all.io/models/models.json
    return gpt4all.GPT4All(os.environ["MODEL_NAME"])


image = modal.Image.debian_slim().env(
    {"MODEL_NAME": os.environ["MODEL_NAME"]}
).pip_install("gpt4all").run_function(download_model)
stub = modal.Stub("gpt4all", image=image)


@stub.cls(keep_warm=1)
class GPT4All:
    def __enter__(self):
        print("Downloading model")
        self.gpt4all = download_model()
        print("Loaded model")

    @modal.method()
    def generate(self, prompt: str):
        messages = [{"role": "user", "content": prompt}]
        completion = self.gpt4all.chat_completion(messages)
        return completion

In [9]:
with stub.run():
    model = GPT4All()
    completion = model.generate.call(prompt="What is AI?")

Output()

✓ Initialized. View app at https://modal.com/apps/ap-Xrx3iOxFUxdclGPchfZJCs

Output()

✓ Created objects.
├── 🔨 Created download_model.
└── 🔨 Created GPT4All.generate.

Output()

...

### Instruction: 
            The prompt below is a question to answer, a task to complete, or a conversation 
            to respond to; decide which and write an appropriate response.
            
### Prompt: 
What is AI?
### Response:


Artificial Intelligence (AI) is the branch of computer science that aims to create intelligent machines. It is the study of how computers can act intelligently. It is the study of how computers can act intelligently.


Exception ignored in: <function blocking___del__ at 0x7f0429d15750>
Traceback (most recent call last):
  File "/pkg/synchronicity/synchronizer.py", line 537, in proxy_method


    instance = self.__dict__[synchronizer_self._original_attr]
KeyError: '_sync_original_139655887705504'


✓ App completed.

In [5]:
completion

{'model': 'ggml-mpt-7b-instruct',
 'usage': {'prompt_tokens': 237,
  'completion_tokens': 217,
  'total_tokens': 454},
 'choices': [{'message': {'role': 'assistant',
    'content': 'Artificial Intelligence (AI) is the branch of computer science that aims to develop intelligent machines. It is the study of how computers can act intelligently. AI is the study of how computers can act intelligently.'}}]}

In [ ]:
# https://modal.com/docs/guide/trigger-deployed-functions
f = modal.Function.lookup("gpt4all", "GPT4All.generate")
completion = f.call(prompt="What is AI?")